# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [21]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [22]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [23]:
# Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [24]:
# Handling missing values - filling numerical columns with median and categorical with mode
spaceship['Age'].fillna(spaceship['Age'].median(), inplace=True)
spaceship['RoomService'].fillna(spaceship['RoomService'].median(), inplace=True)
spaceship['FoodCourt'].fillna(spaceship['FoodCourt'].median(), inplace=True)
spaceship['ShoppingMall'].fillna(spaceship['ShoppingMall'].median(), inplace=True)
spaceship['Spa'].fillna(spaceship['Spa'].median(), inplace=True)
spaceship['VRDeck'].fillna(spaceship['VRDeck'].median(), inplace=True)

In [25]:
spaceship['HomePlanet'].fillna(spaceship['HomePlanet'].mode()[0], inplace=True)
spaceship['CryoSleep'].fillna(spaceship['CryoSleep'].mode()[0], inplace=True)
spaceship['Cabin'].fillna(spaceship['Cabin'].mode()[0], inplace=True)
spaceship['Destination'].fillna(spaceship['Destination'].mode()[0], inplace=True)
spaceship['VIP'].fillna(spaceship['VIP'].mode()[0], inplace=True)
spaceship['Name'].fillna(spaceship['Name'].mode()[0], inplace=True)

In [26]:
# Converting categorical columns to category type
categorical_columns = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Name']
for col in categorical_columns:
    spaceship[col] = spaceship[col].astype('category')

In [27]:
spaceship['Transported'] = spaceship['Transported'].astype('int')


In [28]:
X = spaceship.drop(columns=['Transported', 'PassengerId', 'Name'])
y = spaceship['Transported']

In [29]:
# One-hot encode categorical variables
X = pd.get_dummies(X, drop_first=True)

In [30]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [32]:
# Feature selection using mutual_info_classif
selector = SelectKBest(score_func=mutual_info_classif, k=10)  # Select top 10 features
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)


In [33]:
# Display the selected feature names
selected_features = X.columns[selector.get_support()]
print("Selected features:", selected_features)

Selected features: Index(['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'CryoSleep_True', 'Cabin_E/397/P', 'Cabin_F/1417/P', 'Cabin_F/682/S',
       'Cabin_G/880/S'],
      dtype='object')


In [34]:
# Display the shape of the train and test sets after feature selection
print(f"Training set shape (after feature selection): {X_train_selected.shape}")
print(f"Testing set shape (after feature selection): {X_test_selected.shape}")

Training set shape (after feature selection): (6954, 10)
Testing set shape (after feature selection): (1739, 10)


In [35]:
# Check the first few rows of the scaled and selected training set
print(pd.DataFrame(X_train_selected, columns=selected_features).head())

   RoomService  FoodCourt  ShoppingMall       Spa    VRDeck  CryoSleep_True  \
0    -0.326853  -0.251604     -0.282179  0.312621 -0.256843       -0.725954   
1    -0.326853   0.479133     -0.231124 -0.279427 -0.256843       -0.725954   
2    -0.326853  -0.286859     -0.282179 -0.279427 -0.256843        1.377498   
3    -0.326853  -0.285577      0.193788  0.601424 -0.256843       -0.725954   
4    -0.326853  -0.286859     -0.282179 -0.279427 -0.256843        1.377498   

   Cabin_E/397/P  Cabin_F/1417/P  Cabin_F/682/S  Cabin_G/880/S  
0      -0.011993       -0.011993      -0.011993      -0.011993  
1      -0.011993       -0.011993      -0.011993      -0.011993  
2      -0.011993       -0.011993      -0.011993      -0.011993  
3      -0.011993       -0.011993      -0.011993      -0.011993  
4      -0.011993       -0.011993      -0.011993      -0.011993  


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [37]:
# Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


In [38]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Feature selection using mutual_info_classif
selector = SelectKBest(score_func=mutual_info_classif, k=10)  # Select top 10 features
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

# Bagging with Decision Tree Classifier
bagging_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=100,
    max_samples=0.8, bootstrap=True, random_state=42
)

bagging_clf.fit(X_train_selected, y_train)
y_pred_bagging = bagging_clf.predict(X_test_selected)

In [39]:
# Calculate accuracy
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
print(f"Bagging Accuracy: {accuracy_bagging:.4f}")

Bagging Accuracy: 0.7740


In [40]:
# Pasting with Decision Tree Classifier
pasting_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=100,
    max_samples=0.8, bootstrap=False, random_state=42
)

pasting_clf.fit(X_train_selected, y_train)
y_pred_pasting = pasting_clf.predict(X_test_selected)

# Calculate accuracy
accuracy_pasting = accuracy_score(y_test, y_pred_pasting)
print(f"Pasting Accuracy: {accuracy_pasting:.4f}")

Pasting Accuracy: 0.7723


- Random Forests

In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [42]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Feature selection using mutual_info_classif
selector = SelectKBest(score_func=mutual_info_classif, k=10)  # Select top 10 features
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

# Random Forest with Decision Tree Classifier
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_clf.fit(X_train_selected, y_train)
y_pred_rf = random_forest_clf.predict(X_test_selected)

# Calculate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_rf)
print("Confusion Matrix:\n", conf_matrix)

# Classification Report
class_report = classification_report(y_test, y_pred_rf)
print("Classification Report:\n", class_report)

Random Forest Accuracy: 0.7780
Confusion Matrix:
 [[623 238]
 [148 730]]
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.72      0.76       861
           1       0.75      0.83      0.79       878

    accuracy                           0.78      1739
   macro avg       0.78      0.78      0.78      1739
weighted avg       0.78      0.78      0.78      1739



- Gradient Boosting

In [45]:
# Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [46]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Feature selection using mutual_info_classif
selector = SelectKBest(score_func=mutual_info_classif, k=10)  # Select top 10 features
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_clf.fit(X_train_selected, y_train)
y_pred_gb = gb_clf.predict(X_test_selected)

# Calculate accuracy
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f"Gradient Boosting Accuracy: {accuracy_gb:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_gb)
print("Confusion Matrix:\n", conf_matrix)

# Classification Report
class_report = classification_report(y_test, y_pred_gb)
print("Classification Report:\n", class_report)

Gradient Boosting Accuracy: 0.7780
Confusion Matrix:
 [[603 258]
 [128 750]]
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.70      0.76       861
           1       0.74      0.85      0.80       878

    accuracy                           0.78      1739
   macro avg       0.78      0.78      0.78      1739
weighted avg       0.78      0.78      0.78      1739



- Adaptive Boosting

In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [48]:
# Feature selection using mutual_info_classif
selector = SelectKBest(score_func=mutual_info_classif, k=10)  # Select top 10 features
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

# AdaBoost Classifier with Decision Tree Classifier as the base estimator
ada_clf = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

ada_clf.fit(X_train_selected, y_train)
y_pred_ada = ada_clf.predict(X_test_selected)

# Calculate accuracy
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print(f"AdaBoost Accuracy: {accuracy_ada:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_ada)
print("Confusion Matrix:\n", conf_matrix)

# Classification Report
class_report = classification_report(y_test, y_pred_ada)
print("Classification Report:\n", class_report)

TypeError: AdaBoostClassifier.__init__() got an unexpected keyword argument 'base_estimator'

In [49]:
# Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load dataset
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")

# Handling missing values
spaceship['Age'].fillna(spaceship['Age'].median(), inplace=True)
spaceship['RoomService'].fillna(spaceship['RoomService'].median(), inplace=True)
spaceship['FoodCourt'].fillna(spaceship['FoodCourt'].median(), inplace=True)
spaceship['ShoppingMall'].fillna(spaceship['ShoppingMall'].median(), inplace=True)
spaceship['Spa'].fillna(spaceship['Spa'].median(), inplace=True)
spaceship['VRDeck'].fillna(spaceship['VRDeck'].median(), inplace=True)
spaceship['HomePlanet'].fillna(spaceship['HomePlanet'].mode()[0], inplace=True)
spaceship['CryoSleep'].fillna(spaceship['CryoSleep'].mode()[0], inplace=True)
spaceship['Cabin'].fillna(spaceship['Cabin'].mode()[0], inplace=True)
spaceship['Destination'].fillna(spaceship['Destination'].mode()[0], inplace=True)
spaceship['VIP'].fillna(spaceship['VIP'].mode()[0], inplace=True)
spaceship['Name'].fillna(spaceship['Name'].mode()[0], inplace=True)

# Converting categorical columns to category type
categorical_columns = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Name']
for col in categorical_columns:
    spaceship[col] = spaceship[col].astype('category')

# Convert 'Transported' column to binary
spaceship['Transported'] = spaceship['Transported'].astype('int')

# One-hot encode categorical variables
X = spaceship.drop(columns=['Transported', 'PassengerId', 'Name'])
X = pd.get_dummies(X, drop_first=True)
y = spaceship['Transported']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Feature selection using mutual_info_classif
selector = SelectKBest(score_func=mutual_info_classif, k=10)  # Select top 10 features
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

# AdaBoost Classifier with Decision Tree Classifier as the base estimator
ada_clf = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

ada_clf.fit(X_train_selected, y_train)
y_pred_ada = ada_clf.predict(X_test_selected)

# Calculate accuracy
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print(f"AdaBoost Accuracy: {accuracy_ada:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_ada)
print("Confusion Matrix:\n", conf_matrix)

# Classification Report
class_report = classification_report(y_test, y_pred_ada)
print("Classification Report:\n", class_report)
# Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load dataset
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")

# Handling missing values
spaceship['Age'].fillna(spaceship['Age'].median(), inplace=True)
spaceship['RoomService'].fillna(spaceship['RoomService'].median(), inplace=True)
spaceship['FoodCourt'].fillna(spaceship['FoodCourt'].median(), inplace=True)
spaceship['ShoppingMall'].fillna(spaceship['ShoppingMall'].median(), inplace=True)
spaceship['Spa'].fillna(spaceship['Spa'].median(), inplace=True)
spaceship['VRDeck'].fillna(spaceship['VRDeck'].median(), inplace=True)
spaceship['HomePlanet'].fillna(spaceship['HomePlanet'].mode()[0], inplace=True)
spaceship['CryoSleep'].fillna(spaceship['CryoSleep'].mode()[0], inplace=True)
spaceship['Cabin'].fillna(spaceship['Cabin'].mode()[0], inplace=True)
spaceship['Destination'].fillna(spaceship['Destination'].mode()[0], inplace=True)
spaceship['VIP'].fillna(spaceship['VIP'].mode()[0], inplace=True)
spaceship['Name'].fillna(spaceship['Name'].mode()[0], inplace=True)

# Converting categorical columns to category type
categorical_columns = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Name']
for col in categorical_columns:
    spaceship[col] = spaceship[col].astype('category')

# Convert 'Transported' column to binary
spaceship['Transported'] = spaceship['Transported'].astype('int')

# One-hot encode categorical variables
X = spaceship.drop(columns=['Transported', 'PassengerId', 'Name'])
X = pd.get_dummies(X, drop_first=True)
y = spaceship['Transported']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Feature selection using mutual_info_classif
selector = SelectKBest(score_func=mutual_info_classif, k=10)  # Select top 10 features
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

# AdaBoost Classifier with Decision Tree Classifier as the base estimator
ada_clf = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

ada_clf.fit(X_train_selected, y_train)
y_pred_ada = ada_clf.predict(X_test_selected)

# Calculate accuracy
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print(f"AdaBoost Accuracy: {accuracy_ada:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_ada)
print("Confusion Matrix:\n", conf_matrix)

# Classification Report
class_report = classification_report(y_test, y_pred_ada)
print("Classification Report:\n", class_report)


c:\Users\gianf\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Accuracy: 0.7786
Confusion Matrix:
 [[601 260]
 [125 753]]
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.70      0.76       861
           1       0.74      0.86      0.80       878

    accuracy                           0.78      1739
   macro avg       0.79      0.78      0.78      1739
weighted avg       0.79      0.78      0.78      1739



c:\Users\gianf\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Accuracy: 0.7786
Confusion Matrix:
 [[601 260]
 [125 753]]
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.70      0.76       861
           1       0.74      0.86      0.80       878

    accuracy                           0.78      1739
   macro avg       0.79      0.78      0.78      1739
weighted avg       0.79      0.78      0.78      1739



Which model is the best and why?

In [ ]:
#comment here